# Grouping and Sorting with cuDF

In this notebook you will be introduced to grouping and sorting with cuDF, with performance comparisons to Pandas, before integrating what you learned in a short data analysis exercise.

## Objectives

By the time you complete this notebook you will be able to:

- Perform GPU-accelerated group and sort operations with cuDF

## Imports

In [1]:
import cudf
import pandas as pd

## Read Data

We once again read the UK population data, returning to timed comparisons with Pandas.

In [2]:
%time gdf = cudf.read_csv('./data/pop_1-04.csv', dtype=['float32', 'str', 'str', 'float32', 'float32', 'str'])

CPU times: user 2.09 s, sys: 597 ms, total: 2.68 s
Wall time: 2.68 s


In [3]:
%time df = pd.read_csv('./data/pop_1-04.csv')

CPU times: user 25.8 s, sys: 3.77 s, total: 29.6 s
Wall time: 29.5 s


In [4]:
gdf.dtypes

age       float32
sex        object
county     object
lat       float32
long      float32
name       object
dtype: object

In [5]:
gdf.shape

(58479894, 6)

In [6]:
gdf.head()

,age,sex,county,lat,long,name
0,0.0,m,Darlington,54.533638,-1.524400,Francis
1,0.0,m,Darlington,54.426254,-1.465314,Edward
2,0.0,m,Darlington,54.555199,-1.496417,Teddy
3,0.0,m,Darlington,54.547905,-1.572341,Angus
4,0.0,m,Darlington,54.477638,-1.605995,Charlie


## Grouping and Sorting

### Record Grouping

Record grouping with cuDF works the same way as in Pandas.

#### cuDF

In [10]:
%%time
counties = gdf[['county', 'age']].groupby(['county'])
avg_ages = counties.mean()
print(avg_ages[:5])

                      age
county                   
Warrington      40.888416
Reading         35.868777
Derbyshire      42.913279
East Sussex     44.757385
Northumberland  44.626919
CPU times: user 70.5 ms, sys: 8.57 ms, total: 79.1 ms
Wall time: 77.7 ms


#### Pandas

In [8]:
%%time
counties_pd = df[['county', 'age']].groupby(['county'])
avg_ages_pd = counties_pd.mean()
print(avg_ages_pd[:5])

                                    age
county                                 
Barking And Dagenham          33.056845
Barnet                        37.629770
Barnsley                      41.201061
Bath And North East Somerset  39.822837
Bedford                       39.715300
CPU times: user 3.95 s, sys: 973 ms, total: 4.93 s
Wall time: 4.9 s


## Sorting

Sorting is also very similar to Pandas, though cuDF does not support in-place sorting.

#### cuDF

In [11]:
%time gdf_names = gdf['name'].sort_values()
print(gdf_names[:5]) # yes, "A" is an infrequent but correct given name in the UK, according to census data
print(gdf_names[-5:])

CPU times: user 1.67 s, sys: 5.52 ms, total: 1.68 s
Wall time: 1.68 s
26850     A
154537    A
165578    A
211428    A
236972    A
Name: name, dtype: object
58060377    Zyrah
58289490    Zyrah
58363665    Zyrah
58388727    Zyrah
58394184    Zyrah
Name: name, dtype: object


#### Pandas

This operation takes a while with Pandas. Feel free to start the next exercise while you wait.

In [12]:
%time df_names = df['name'].sort_values()
print(df_names[:5])
print(df_names[-5:])

CPU times: user 1min 43s, sys: 1.55 s, total: 1min 45s
Wall time: 1min 45s
10811041    A
17931460    A
5060367     A
1842288     A
24866365    A
Name: name, dtype: object
47008072    Zyrah
47953653    Zyrah
31838209    Zyrah
53669567    Zyrah
54557840    Zyrah
Name: name, dtype: object


## Exercise: Youngest Names

For this exercise you will need to use both `groupby` and `sort_values`.

We would like to know which names are associated with the lowest average age and how many people have those names. Using the `mean` and `count` methods on the data grouped by name, identify the three names with the lowest mean age and their counts.

In [20]:
%%time
names = gdf[['name', 'age']].groupby(['name'])
ages = names.mean()
counts = names.count()
counts

CPU times: user 115 ms, sys: 24 ms, total: 140 ms
Wall time: 138 ms


,age
name,
Nea,286
Remel,356
Fionn,4372
Yannick,508
Dantae,329
...,...
Jaxson,35328
Colby,4820
Larysa,289


In [30]:
ages_counts = cudf.DataFrame()
ages_counts
ages_counts['mean_age'] = ages
ages_counts['count'] = counts
ages_counts = ages_counts.sort_values('mean_age')
ages_counts.iloc[:3]

,mean_age,count
name,,
Leart,34.911197,259
Luke-Junior,35.313725,255
Nameer,35.479675,246


#### Solution

In [18]:
# %load solutions/youngest_names
name_groups = gdf[['name', 'age']].groupby('name')

name_ages = name_groups['age'].mean()
name_counts = name_groups['age'].count()

ages_counts = cudf.DataFrame()
ages_counts['mean_age'] = name_ages
ages_counts['count'] = name_counts

ages_counts = ages_counts.sort_values('mean_age')
ages_counts.iloc[:3]


<br>
<div align="center"><h2>Please Restart the Kernel</h2></div>

In [31]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

## Next

As part of our larger data science goal for this workshop, we will be working with data reflecting the entire road network of Great Britain. In the next notebook you will be exposed to additional cuDF techniques that you will use to transform columnar data into graph edge data that we will be using to construct a GPU-accelerated graph using the `cuGraph` library.